# Road Following Data Collection - PLUS v2.0
Author: George Gorospe

Note: this code draws heavily on the NVIDIA data_collection.ipynb notebook shipped with the Jetbot sold by Sparkfun.com.
You can find the original code here: https://github.com/NVIDIA-AI-IOT/jetbot

Here, our goal is to enable our Jetbot to follow a road designated by a lane or limits on the left and right of the road. This is similar to line following. However, unlike line following, no lines are made for the robot, the Jetbot will now have to adapt to its environment.

Essentially you'll be teaching your Jetbot how to drive while staying inside the limits of the road or path. To do this, lets consider what type of data we need to collect...

We want our Jetbot to know how to steer given its position on the road or track.
So we'll need many photos and steering angles guiding the robot to a safe trajectory or path at various points in the road.

In this notebook, we are going to control the Jetbot, commanding it to move along the road, while we collect data (photos) and viable path data (x and y coordinates). That can be later used to train a machine learning model. In this case, we're training the model to steer for us. Let's get started!

Road Following
If you've run through the collision avoidance sample, your should be familiar following three steps

Data collection
Training
Deployment
In this notebook, we'll do the same exact thing! Except, instead of classification, you'll learn a different fundamental technique, regression, that we'll use to enable JetBot to follow a road (or really, any path or target point).

Place the JetBot in different positions on a path (offset from center, different angles, etc)
Remember from collision avoidance, data variation is key!

Display the live camera feed from the robot
Using a gamepad controller, place a 'green dot', which corresponds to the target direction we want the robot to travel, on the image.
Store the X, Y values of this green dot along with the image from the robot's camera
Then, in the training notebook, we'll train a neural network to predict the X, Y values of our label. In the live demo, we'll use the predicted X, Y values to compute an approximate steering value (it's not 'exactly' an angle, as that would require image calibration, but it's roughly proportional to the angle so our controller will work fine).

So how do you decide exactly where to place the target for this example? Here is a guide we think may help

Assuming our deep learning model works as intended, these labeling guidelines should ensure the following:

The robot can safely travel directly towards the target (without going out of bounds etc.)
The target will continuously progress along our imagined path
What we get, is a 'carrot on a stick' that moves along our desired trajectory. Deep learning decides where to place the carrot, and JetBot just follows it :)

### Labeling example video
Execute the block of code to see an example of how to we labeled the images. This model worked after only 123 images :)

Look at the live video feed from the camera
Imagine the path that the robot should follow (try to approximate the distance it needs to avoid running off road etc.)
Place the target as far along this path as it can go so that the robot could head straight to the target without 'running off' the road.
For example, if we're on a very straight road, we could place it at the horizon. If we're on a sharp turn, it may need to be placed closer to the robot so it doesn't run out of boundaries.

In [ ]:
from IPython.display import HTML
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/FW4En6LejhI" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

### Import Required Libraries

The following python libraries are required for the data collection process.
If you get an error here, you probably need to install the clickable image widget.
Follow the instructions here: https://github.com/jaybdub/jupyter_clickable_image_widget
 

In [1]:
# IPython Libraries for display and widgets
import traitlets
import ipywidgets
import ipywidgets.widgets as widgets
from IPython.display import display
from jupyter_clickable_image_widget import ClickableImageWidget


# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Python basic packages for image annotation
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

os.system('systemctl restart nvargus-daemon')

256

### Robot Control Functions

To start the data collection process, we first need to control the movement of the robot and drive it around the roads we want it to follow. These functions are an easy way to control the robot. They utilize parameters for speed and sleep duration

In [2]:
from jetbot import Robot
import time

# Robot object
robot = Robot()

# Our robot motion control functions, they use speed, direction, and duration.
def robotLeft():
    robot.left(speed=0.5)
    time.sleep(0.3)
    robot.stop()

def robotRight():
    robot.right(speed=0.5)
    time.sleep(0.3)
    robot.stop()
    
def robotForward():
    robot.forward(speed=0.5)
    time.sleep(0.3)
    robot.stop()

def robotBackward():
    robot.backward(speed=0.5)
    time.sleep(0.3)
    robot.stop()

# Sometimes the robot can start moving when these functions are defined if the functions were executed previously. This command is here to ensure that the robot behaves.
robot.stop()

### Data Collection

There is a lot going on in the next block of code, so let's break it down.
We need to create a folder to hold the data we collect.
That data will be photos and unlike pervious data collection the filenames for each photo is now important. This is because each photo will have two elements: 
1. An image of the track/road/path
2. The best path to take in the current situation, represented by x and y coordiantes.

This means the the file names will ahve the following structure:
``xy_<x value>_<y value>_<uuid>.jpg``

This structure will be used when training the machine learning model.

When we train, we load the images and parse the x, y values from the filename.

Creating and Using Widgets: widgets are incredibly useful. To use them you first create them, then attach call back functions, and finally display the widget for the user.

In this case we use widgets for data collection and control of the Jetbot

Instructions:
1. Drive your Jetbot around the path.
2. At various positions, both on the best possible path and pointing incorrectly towards the edge of the path, click within the image on a path you think the robot should take given the current position.

Note: take a wide varitey of data points, not just the best case but also cases in which the Jetbot has turned too much or too little and is in dager of hitting a wall or going outside of the path/road.

In [3]:
DATASET_DIR = 'dataset_xy'

# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created because they already exist')

# Setup the camera and create a widget to view what the Jetbot sees.
camera = Camera()
camera_widget = ClickableImageWidget(width=camera.width, height=camera.height)
snapshot_widget = ipywidgets.Image(width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# Widget Creation: sliders and control buttons
button_layout = widgets.Layout(width='128px', height='64px')
leftButton = widgets.Button(description='LEFT', button_style='success', layout=button_layout)
rightButton = widgets.Button(description='RIGHT', button_style='success', layout=button_layout)
forwardButton = widgets.Button(description='FORWARD', button_style='success', layout=button_layout)
backwardButton = widgets.Button(description='BACKWARD', button_style='success',layout=button_layout)

# Attach callbacks for Jetbot control, these reference to the functions we created earlier
leftButton.on_click(lambda x: robotLeft())
rightButton.on_click(lambda x: robotRight())
forwardButton.on_click(lambda x: robotForward())
backwardButton.on_click(lambda x: robotBackward())

# A callback for the count textbox widget
count_widget = widgets.IntText(description='count', value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))))

# Creating two new functions used for collecting photos and naming them.
def xy_uuid(x, y):
    return 'xy_%03d_%03d_%s' % (x * 50 + 50, y * 50 + 50, uuid1())

def save_snapshot(_, content, msg):
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        
        # save to disk
        #dataset.save_entry(category_widget.value, camera.value, x, y)
        uuid = 'xy_%03d_%03d_%s' % (x, y, uuid1())
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(camera_widget.value)
        
        # display saved snapshot
        snapshot = camera.value.copy()
        snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))
        
camera_widget.on_msg(save_snapshot)

# Setting up the widgets for display and control
data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget, snapshot_widget]),
    count_widget
])

jetbot_control_widget = ipywidgets.VBox([ipywidgets.HBox([leftButton, forwardButton, backwardButton ,rightButton])
])

# Now that we've created the widgets and attached callback functions to each of them
# we can display the widgets and create the user interface for our Jetbot
display(data_collection_widget)

display(jetbot_control_widget)



Directories not created because they already exist
